In [1]:
import json
from pprint import pprint


In [98]:

import pandas as pd
import numpy as np
from datetime import datetime
from enum import Enum
from abc import ABC, abstractmethod

In [99]:
#a = open('feeddata_10.json')


In [100]:
#'KR is for Korea
#4 for derivatives
#'Then type 1 for future ,2 for call, 3 for put
#'0 no clue
#'Then 1 for maxi or 5 for mini
#'Then 2 letters expiry code, 1 one is year, second month
#'Then 3 digits for the strike
#'Then 1 for isin checksum


In [101]:
data = json.load(open('feeddata_10.json'))

In [102]:
#aPD = pd.DataFrame.from_records(data)
#books = aPD.book.drop_duplicates().str.split('01')
#books = aPD.book.drop_duplicates() #check 1 if maxi 5 if mini
#aTest = books.to_frame

In [103]:
class CONTRACTTYPE(Enum):
    UNSET = 0
    FUTURE = 1
    CALL = 2
    PUT = 3
    
#class Strike(Enum):
#    for i in range(50)+100:
#         str(i+2.5*i) = i-99

In [104]:
class Books():
    
    contractType = CONTRACTTYPE.UNSET
    book = []

    
    def __init__(self,*args):
        self.contractType = args[0]
        if args[1]:
            self.strike = args[1]
        
    def setStrike(aStrike):
        self.strike = aStrike

    def updateBook(aTickBook):
        self.book = aTickBook
        self.updateTime = ''
        self.timeToExpiry = ''
        
    def getVolume(level, isBid):
        return 0
        
    def getBestBid():
        return 0
    
    def getBestAsk():
        return 0

In [105]:
class aSimpleBook():
    
    aStrikeBook = {}
    
    def getBestBid_Price(aStrike):
        return aStrikeBook["aStrike"]['bid_price_0']
    
    def getBestAsk_Price(aStrike):
        return aStrikeBook["aStrike"]['ask_price_0']
    
    def getBestBid_Volume(aStrike):
        return aStrikeBook["aStrike"]['bid_volume_0']
    
    def getBestAsk_Volume(aStrike):
        return aStrikeBook["aStrike"]['ask_volume_0']
    
    def updateBook(aNewTick):
        aStrikeBook["aStrike"] = aNewTick
        


In [106]:
class Traders(ABC):


    @abstractmethod
    def getPnL(self):
        pass
    @abstractmethod
    def getPosition(self):
        pass
    @abstractmethod
    def update(self):
        pass

In [107]:
class TraderTest(Traders):
    Position = {}
    LastFutureValue = 0
    PnL = 0
    theStrategies = []
    theBook = aSimpleBook
    
    def __init__(self, aBunchOfStrategies):
        theStrategies = aBunchOfStrategies
    
    def getPnL(self):
        return self.PnL
    
    def getPosition(self, strike): #options
        return self.Position[1:]
    
    def getPosition(self):  #futures
        return self.Position[0]
    
    def updatePnL(self):
        return 0
    
    def update(self,aTick):
        
        theBook.update(aTick["rawStrikeBook"])
            
        for strat in theStrategies:
            Position[aTick["Strike"]] = strat.getAction(aTick, theBook)
        
        
        

In [108]:
class Strategy(ABC):
    
    @abstractmethod
    def getAction(self,aTick):
        return 0

In [109]:
class StrategyTest():
    
    def getAction(aTick, LastFutureValue, aBook):
        
        myQty = 0 
        
        if(aTick["contract"] == CONTRACTTYPE.CALL):
            if aTick["midPoint"] > LastFutureValue - float(aTick["Strike"])*getDiscount(aTick) + putPrice:
                myQty = -1 
                
                
        return myQty
        
        
    
    
    

In [110]:
def isCorrectISINCHECK(aISIN):
    #TODO
    return True

In [111]:
def getDiscount(aTick):
    
    rate = 0.025
    
    return np.exp( -rate * (aTick["timeToExpiry"].days + aTick["timeToExpiry"].seconds / (60*60*24))/365)
    
    

In [112]:
import calendar
import sys
from dateutil import tz

def getExpiryDate(anExpiryCode):
    #hardcoded version for now. 
    myString_date = ''
    
    myString_year = ''
    myString_month = ''
    myString_day = ''
    
    #get year code
    if anExpiryCode[0] == 'L':
        myString_year = "2016"
    elif anExpiryCode[0] == 'M':
        myString_year = "2017"
    elif anExpiryCode[0] == 'N':
        myString_year = "2018"
    elif anExpiryCode[0] == 'P': #It looks strange, but that's KRX 
        myString_year = "2019"
    else : 
        sys.stderr.write("fatal error\n expiry year code unknown \n")
    
    #get month code
    myPotentialMonths = ['','1','2','3','4','5','6','7','8','9','A','B','C']

    if anExpiryCode[1] in myPotentialMonths:
        myString_month =  str(myPotentialMonths.index(anExpiryCode[1]))
    else : 
        sys.stderr.write("fatal error\n expiry month code unknown \n")    
    
    if myString_month == '':
        sys.stderr.write("fatal error\n expiry month code unknown \n")   
    #get day
    #get third Wednesday of every month, we will have to make bank holidays exceptions
    myString_day = str(calendar.Calendar(2).monthdatescalendar(int(myString_year), \
                                                               int(myString_month))[3][0].day)
    
                       
    myString_date = myString_year+'-'+myString_month+'-'+myString_day+' 16:00:00'

    timeZone = tz.gettz('Asia/Seoul')
            
    #print("Don't forget bank holidays exceptions")
    return datetime.strptime(myString_date, "%Y-%m-%d %H:%M:%S").replace(tzinfo=timeZone).astimezone(timeZone)


In [113]:

def utcToRightTMZ(aUTCTMZ):
    
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('Asia/Seoul')
    
    aUTCTMZ = aUTCTMZ.replace(tzinfo=from_zone)
    localTimeStamp = aUTCTMZ.astimezone(to_zone)
    
    return localTimeStamp

In [116]:
def getBookContract(aStringNumberContract, aTypeOption):
    aContract = ''
    if aStringNumber == "000" and aTypeOption == "1":
        aContract = CONTRACTTYPE.FUTURE #replace with enums
    elif aTypeOption == "2":
        aContract = CONTRACTTYPE.CALL #replace with enums
    elif aTypeOption == "2":
        aContract = CONTRACTTYPE.CALL #replace with enums
    else:
        print("error, Contract Type" + str(aStringNumberContract) + " , " + str(aTypeOption) + " not recognized")
    
    return aContract

In [ ]:
class simulationCrunsher():
    
    def run():
        counter = 0
        myFirstLastDone = True

        for tick in data:
            counter = counter + 1 
            myTick = dict()

            myBook = tick["book"]
            if not isCorrectISINCHECK(myBook):
                sys.stderr.write("fatal error\n ISIN CHECK IS BREAKING \n")   

            if myBook[5] == '1' and  tick['type']=='tick':   #let's forget about mini futures
                try:
                    #print(myBook)
                    myTick["contract"] = getBookContract(myBook[8:11],myBook[3])

                    timeStamp = utcToRightTMZ(datetime.utcfromtimestamp(tick["created"]/1000000))
                    #print(timeStamp)
                    myTick["created"] = timeStamp
                    myTick["received"] = utcToRightTMZ(datetime.utcfromtimestamp(tick["received"]/1000000))

                    myTempStrike = data[1]['book'][8:11]

                    if(myTick["contract"]  == CONTRACTTYPE.OPTION):
                        myTick["Strike"] = myBook[8:11]
                    #print(myBook[6:8])
                    myTick["expiry"] = getExpiryDate(myBook[6:8])

                    myTick["spread"] = (tick['ask_price_0']-tick['bid_price_0'])/2
                    myTick["midPoint"] =(tick['ask_price_0']+tick['bid_price_0'])/2

                    myTick["timeToExpiry"] = myTick["expiry"] - myTick["received"]
                    
                    myTick["rawStrikeBook"] = tick
                    
                except Exception as e: 
                    print(e)
                    print(tick)
                    break

            elif myBook[5] == '1' and tick['type']=='lastdone':
                if myFirstLastDone: 
                    print("@TODO : Handle Last Done")
                myFirstLastDone = False
            elif myBook[5] == '1':
                print(tick)

            if counter > 2000 :
                break

example:

In [95]:
myTraderTest = TraderTest(simulationCrunsher)

TypeError: Can't instantiate abstract class TraderTest with abstract methods runTick